# Final Submission Case Study 1

This notebook is my final submission for case study 1 from a deployment point of view. As discussed, this will focus on a new and more practical Deployment approach for this project. 

In this approach, I am only taking certain useful data from the user: specifically has current application data and a txt file upload for all his previous application data. 

Based on a pretrained model I will be making predictions on the likeliness that a user is a defaulter. 

Hence, this notebook is divided into 2 parts:

1. Model training for this approach. We will load the data, pre process the data, create new features based on Feature Engineering and finally train a model. 

For this notebook, I have chosen to go with testing the XGBoost and Lightgbm model as they performed best during my intial testing on the whole data.

2. After the model training and file saving is comeplete, the next step would be to create a final submission in the given format. I will be creating 2 functions. 

Function 1 will take all the input data given as a single data point and output a prediction value.

Function 2 will take in the input datapoint and the known target value and output a metric score, which is AUC Score in this case. 


# Part 1 - Data Preperation, FE and Model Training

In [ ]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11687 sha256=66d813547ec34dfafe0385fe8bf2c93de8f578dc083591aacba41eb130a464e5
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
# importing the required libraries
import pandas as pd
import numpy as np
import lightgbm as lgbm
import xgboost as xgb
import matplotlib.pyplot as plt
import tqdm
import os
from time import time
import itertools
import gc
from bayes_opt import BayesianOptimization
from scipy.sparse import hstack

# Sklearn Imports
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import recall_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Downloading and loading the data into disk
!wget wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,hi;q=0.6" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/9120/860599/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1617701980&Signature=jZEuiUCD%2B%2FrSm3nJq5rSr10wK2nsJyIUcRxixH4pEy08y2O%2FlsSbEqrHxY44F8RJ%2Bf8tYeJ%2Bx8lw%2Fx%2Brk017uPrxOtczQq2iyqqJ0PlsHV%2FcHLuj060fGqthcsbfRIq14zZRMr1wrWEJXSWrBxMU20KRMeanWwLg4jgek3PoRR9u8uj720Mk4f9U50OlABb%2B3Wflk34zAexgD2QLviahtlNxQ9jpfxy8ao7edT7nD90Bft6U9UAoGX9lwOFOWX8SJCV%2BJDbXqTYQj%2FMMwrsvyqa7tHjo85g%2BK4%2BJUWzfA39irFiM063A3s0nghOPQD5qcjeEsxBb%2F9FJSI416S0yaA%3D%3D&response-content-disposition=attachment%3B+filename%3Dhome-credit-default-risk.zip" -c -O 'home-credit-default-risk.zip'

print('-'*100)
print('Data Downloaded! Unzipping......')
print('-'*100)
print()

# Once zip file is downloaded, just unzip and store the contents in the '/data' folder
!unzip 'home-credit-default-risk.zip' -d '/data'

--2021-04-05 09:02:14--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2021-04-05 09:02:15--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/9120/860599/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1617701980&Signature=jZEuiUCD%2B%2FrSm3nJq5rSr10wK2nsJyIUcRxixH4pEy08y2O%2FlsSbEqrHxY44F8RJ%2Bf8tYeJ%2Bx8lw%2Fx%2Brk017uPrxOtczQq2iyqqJ0PlsHV%2FcHLuj060fGqthcsbfRIq14zZRMr1wrWEJXSWrBxMU20KRMeanWwLg4jgek3PoRR9u8uj720Mk4f9U50OlABb%2B3Wflk34zAexgD2QLviahtlNxQ9jpfxy8ao7edT7nD90Bft6U9UAoGX9lwOFOWX8SJCV%2BJDbXqTYQj%2FMMwrsvyqa7tHjo85g%2BK4%2BJUWzfA39irFiM063A3s0nghOPQD5qcjeEsxBb%2F9FJSI416S0yaA%3D%3D&response-content-disposition=attachment%3B+filename%3Dhome-credit-default-risk.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.103.128, 142.250.128.128, 142.251.6.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.1

In [ ]:
def one_hot_encoder(df, nan_category = True):
    # List of all original Columns in the dataframe
    original_columns = list(df.columns)
    # List of all categorical features
    categorical_columns = list(df.select_dtypes('object').columns)
    # Doing one-hot encoding for categorical features
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_category)
    # Creating a list of new column names for better reference. 
    new_columns = [col for col in df.columns if col not in original_columns]
    return df, new_columns

In [ ]:
# This class will be used to train a LGBM and XGBOOST model for deployment
class deployment_model_training:
  '''
  This class helps you train a model for deployment. 
  It contains the following functions:
    1. init (initialization)
    2. load_data
    3. preprocessing
    4. XGB_Model
    5. LGB_Model
    6. save_pickles
  '''

  # Function 1 : Initialization
  def __init__(self, file_directory = ''):
    '''
        This function is used to initialize the Class members. 
        
        Inputs:
            self
            file_directory: str, default = ''
                Path of the directory where the file is stored
        
        Returns:
            None
    '''
    self.file_directory = file_directory
  
  # Function 2 : Load Data
  def load_data(self, verbose = False):
    '''
    This functions helps in loading all the required data for model training. 
    In this case, it will load the application train, test and previous application data.

    Inputs: 
      self
    
    Returns:
      None
    '''
    if verbose:
      print('Please wait: Loading the data into memory...')
      start_time = time()

    # Loading the 3 datafiles into memory. 
    self.app_train = pd.read_csv(self.file_directory + 'application_train.csv')
    self.app_test = pd.read_csv(self.file_directory + 'application_test.csv')
    self.prev_app = pd.read_csv(self.file_directory + 'previous_application.csv')

    # mandatory print statement
    print('Data successfully loaded!')

    if verbose:
      print('It took {} seconds to load the data.'.format(round(time() - start_time, 2)))
  
  # Function 3 : Data Pre processing
  def preprocessing(self, verbose = False):
    '''
    This function preprocesses the loaded data. The processing includes:
      1. Choosing only model specific features (already chosen)
      2. Handling Outliers and missing values 
      3. Categorical features encoding (one-hot)
      4. Feature Engineering to create some better features

    Inputs:
      self
      verbose : Whether to print statements (default : False)
    
    Returns:
      train_df : The final processed data ready for model training
      test_df : The final processed data ready for model testing (without target value)
    '''

    # Task 1 : Choose only the important features
    # --------------------------------------------------------------------------------------------------------------
    # i) Application Train, Test
    chosen_features_app = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'CODE_GENDER',
                           'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'AMT_INCOME_TOTAL',
                           'CNT_FAM_MEMBERS', 'FLAG_OWN_CAR', 'OWN_CAR_AGE', 'CNT_CHILDREN',
                           'SK_ID_CURR', 'TARGET']
    
    # ii) Previous Application
    chosen_features_prev = ['AMT_APPLICATION', 'AMT_CREDIT','AMT_DOWN_PAYMENT','AMT_GOODS_PRICE', 'AMT_ANNUITY',
                            'HOUR_APPR_PROCESS_START', 'RATE_DOWN_PAYMENT','DAYS_DECISION', 'CNT_PAYMENT',
                            'NAME_CONTRACT_STATUS', 'AMT_GOODS_PRICE', 'SK_ID_CURR', 'SK_ID_PREV']
    
    # Only get the chosen columns from the data

    deployment_app_train = self.app_train[chosen_features_app]

    # Removing the Target variable
    chosen_features_app.remove('TARGET')

    deployment_app_test = self.app_test[chosen_features_app]
    deployment_prev_app = self.prev_app[chosen_features_prev]

    # --------------------------------------------------------------------------------------------------------------
    
    # Task 2 : Handling missing values and outliers
    # --------------------------------------------------------------------------------------------------------------

    if verbose:
        print('Handling Missing values and outliers......')
    
    # i) Application Train/Test Data

    start = time()
    # Merging train and test data for easier processing
    merged_data = pd.concat([deployment_app_train, deployment_app_test], axis = 0, ignore_index = True)

    # First, removing the XNA gender category. It only has 4 occurences. 
    merged_data.drop(merged_data[merged_data['CODE_GENDER'] == 'XNA'].index, inplace = True)
    # 1. A specific value for days employed. (gives you around 1000 years, which is not possible)
    merged_data['DAYS_EMPLOYED'].replace(365243, np.nan, inplace = True)
    # 4. AMT_INCOME Total greater than 10,000,000 (10 million) is highly unlikely (to default or in general). So, replace it
    merged_data.loc[merged_data['AMT_INCOME_TOTAL'] > 1e7, 'AMT_INCOME_TOTAL'] = np.nan


    # ii) Previous Application Data

    deployment_prev_app.loc[deployment_prev_app['AMT_CREDIT'] > 6000000, 'AMT_CREDIT'] = np.nan

    # --------------------------------------------------------------------------------------------------------------

    # Task 3 : Categorical One Hot encoding
    # --------------------------------------------------------------------------------------------------------------

    # One hot encoding categorical features for application data
    merged_data, merged_data_cat_columns = one_hot_encoder(merged_data)

    # one hot encoding for categorical features, if any for previous application data
    deployment_prev_app, prev_app_cat_features = one_hot_encoder(deployment_prev_app)

    if verbose:
      print('Done with pre processing! Time Elapsed : {} seconds'.format(round(time() - start, 2)))
    # --------------------------------------------------------------------------------------------------------------

    # Task 4 : Feature Engineering
    # --------------------------------------------------------------------------------------------------------------

    start = time()
    if verbose: 
      print('Creating new and better features......')

    # i) Application_train/test Feature Engineering
    merged_data['NEW_CREDIT_TO_ANNUITY_RATIO'] = merged_data['AMT_CREDIT'] / merged_data['AMT_ANNUITY']
    merged_data['NEW_CREDIT_TO_GOODS_RATIO'] = merged_data['AMT_CREDIT'] / merged_data['AMT_GOODS_PRICE']
    merged_data['AGE'] =  pd.to_numeric((merged_data['DAYS_BIRTH'] / -365), downcast='integer')
    merged_data['YEARS_EMPLOYED'] =  pd.to_numeric((merged_data['DAYS_EMPLOYED'] / -365), downcast='integer')
    merged_data['INCOME_PER_PERSON'] = merged_data['AMT_INCOME_TOTAL'] / merged_data['CNT_FAM_MEMBERS']
    merged_data['ANNUITY_INCOME_PERC'] = merged_data['AMT_ANNUITY'] / merged_data['AMT_INCOME_TOTAL']
    merged_data['INCOME_CREDIT_PERC'] = merged_data['AMT_INCOME_TOTAL'] / merged_data['AMT_CREDIT']
    merged_data['DAYS_EMPLOYED_PERC'] = merged_data['DAYS_EMPLOYED'] / merged_data['DAYS_BIRTH']
    merged_data['PAYMENT_RATE'] = merged_data['AMT_ANNUITY'] / merged_data['AMT_CREDIT']

    merged_data.drop(columns = ['DAYS_BIRTH', 'DAYS_EMPLOYED'], inplace = True)
    # ii) Previous Application Data
    deployment_prev_app['APP_CREDIT_PERC'] = deployment_prev_app['AMT_APPLICATION'] / deployment_prev_app['AMT_CREDIT']

    # Aggregations we will perform 
    prev_app_num_agg = {
                    'AMT_ANNUITY': ['min', 'max', 'mean'],
                    'AMT_APPLICATION': ['min', 'max', 'mean'],
                    'AMT_CREDIT': ['min', 'max', 'mean'],
                    'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
                    'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
                    'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
                    'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
                    'DAYS_DECISION': ['min', 'max', 'mean'],
                    'CNT_PAYMENT': ['mean', 'sum']
                   }

    # Perform the aggregations based on the sk_id_curr
    prev_agg_features = deployment_prev_app.groupby('SK_ID_CURR').agg(prev_app_num_agg)
    prev_agg_features.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg_features.columns.tolist()])

    # Previous Applications: Approved Applications - only numerical features
    approved_applications = deployment_prev_app[deployment_prev_app['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg_features = approved_applications.groupby('SK_ID_CURR').agg(prev_app_num_agg)
    approved_agg_features.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg_features.columns.tolist()])
    prev_agg_features = prev_agg_features.join(approved_agg_features, how='left', on='SK_ID_CURR')
    
    # Previous Applications: Refused Applications - only numerical features
    refused_applications = deployment_prev_app[deployment_prev_app['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg_features = refused_applications.groupby('SK_ID_CURR').agg(prev_app_num_agg)
    refused_agg_features.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg_features.columns.tolist()])
    prev_agg_features = prev_agg_features.join(refused_agg_features, how='left', on='SK_ID_CURR')

    del refused_applications, refused_agg_features, approved_applications, approved_agg_features, deployment_prev_app
    gc.collect()

    if verbose:
      print('Done with Feature Engineering! Time Elapsed : {} seconds'.format(round(time() - start, 2)))
    
    # Getting the Final train and test data ready
    print(merged_data.shape)
    print(prev_agg_features.shape)

    # i) Merging both data files
    final_df = merged_data.merge(prev_agg_features, how = 'left', on = 'SK_ID_CURR')

    # ii) Dividing them into train and test data
    train_df = final_df[final_df['TARGET'].notnull()]
    test_df = final_df[final_df['TARGET'].isnull()]

    # Remove the SK_ID_CURR Parameter
    _ = train_df.pop('SK_ID_CURR')
    _ = test_df.pop('SK_ID_CURR')
    
    if verbose:
      print('The shape of Final training data : {}'.format(train_df.shape))
      print('The shape of Final test data : {}'.format(test_df.shape))
    
    return train_df, test_df

  # -------------------------------------------------------------------------------------------------------------

  #---------------------------------------------------------------------------------------------------------------
  # Function 4 : Light GBM Model training
  def lgbm_model(self, train_df, test_df, nfolds = 3):
    '''
    This function trains a Light GBM Model.

    Inputs:
      self
      train : Train data (includes TARGET)
      test : Test Data
      nfolds : Number of folds (default : 3)
    
    output: 
      trained model
    '''

    # Parameters for our LGBM Classifier
    lgbm_params = {
                'njobs': -1,
                'n_estimators': 10000,
                'learning_rate': .02,
                'num_leaves': 34,
                'colsample_bytree': .9497036,
                'subsample': .8715623,
                'max_depth': 7,
                'reg_alpha': .041545473,
                'reg_lambda': .0735294,
                'min_split_gain': .0222415,
                'min_child_weight': 39.3259775,
                'silent': -1,
                'verbose': -1
    }
    fold = StratifiedKFold(n_splits = nfolds, shuffle = True)
    a = 0

    importances = pd.DataFrame()
    y = train_df.pop('TARGET')

    X = train_df
    _ = test_df.pop('TARGET')
    X_test = test_df

    train_predict = np.zeros(train_df.shape[0])
    cv_predict = np.zeros(train_df.shape[0])
    test_predict = np.zeros(test_df.shape[0])

    for i,(train, cv) in enumerate(fold.split(X,y)):
      X_train, Y_train = X.iloc[train], y.iloc[train]
      X_valid, Y_valid = X.iloc[cv], y.iloc[cv]

      model = lgbm.LGBMClassifier(**lgbm_params)
      
      model.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
      
      a=a+1

      train_predict[train] = model.predict_proba(X_train, num_iteration=model.best_iteration_)[:, 1]
      cv_predict[cv]=model.predict_proba(X_valid, num_iteration=model.best_iteration_)[:, 1]
      
      importances["imp"] = model.feature_importances_
      importances["fold"] = i + 1
      print('Fold ',i + 1,' TRAIN AUC : ',roc_auc_score(Y_train, train_predict[train]))
      print('Fold ',i + 1,' CV AUC : ',roc_auc_score(Y_valid, cv_predict[cv]))
            
      del X_train, Y_train, X_valid, Y_valid
      gc.collect()

    # return a trained model
    return model

  def xgb_model(self, train, test, nfolds = 3):
    '''
    This function trains a Light GBM Model.

    Inputs:
      self
      train : Train data (includes TARGET)
      test : Test Data
      nfolds : Number of folds (default : 3)
    
    output: 
      trained model
    '''
    fold = StratifiedKFold(n_splits = nfolds, shuffle = True)
    a = 0

    importances = pd.DataFrame()
    y = train_df.pop('TARGET')

    X = train_df
    _ = test_df.pop('TARGET')
    X_test = test_df

    train_predict = np.zeros(train_df.shape[0])
    cv_predict = np.zeros(train_df.shape[0])
    test_predict = np.zeros(test_df.shape[0])

    for i,(train, cv) in enumerate(fold.split(X,y)):
      X_train, Y_train = X.iloc[train], y.iloc[train]
      X_valid, Y_valid = X.iloc[cv], y.iloc[cv]

      model = xgb.XGBClassifier(learning_rate =0.01, n_estimators=10000, max_depth=4, min_child_weight=5, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', 
            nthread=4, scale_pos_weight=2.5, seed=27, reg_lambda = 1.2, verbose = 5, tree_method = 'gpu_hist')
      
      model.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
      
      a=a+1

      train_predict[train] = model.predict_proba(X_train)[:, 1]
      cv_predict[cv]=model.predict_proba(X_valid)[:, 1]
      
      importances["imp"] = model.feature_importances_
      importances["fold"] = i + 1
      print('Fold ',i + 1,' TRAIN AUC : ',roc_auc_score(Y_train, train_predict[train]))
      print('Fold ',i + 1,' CV AUC : ',roc_auc_score(Y_valid, cv_predict[cv]))
            
      del X_train, Y_train, X_valid, Y_valid
      gc.collect()

    # return a trained model
    return model

    # --------------------------------------------------------------------------------------------------------

    # Function : Create a Submission file
    def submission_file(self, model, file_name):
      '''
        This function creates a submission csv file for a particular model.

        Inputs:
          self
          model : The model for with submission file needs to be created
          file_name : With what name should the csv file be stored

        Returns:
          None
      '''
      # Creating and storing the submission file for test data for all the features
      X_test = test_df
      X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
      X_test = X_test.where(X_test.values <= np.finfo(np.float64).max)
      X_test.fillna(X_test.median(), inplace = True)

      y_pred_lgb = model.predict_proba(X_test)[:, 1]
      submit_lgb = pd.read_csv(directory + 'sample_submission.csv')
      submit_lgb['TARGET'] = y_pred_lgb
      submit_lgb.to_csv(file_name, index = False)

  #-----------------------------------------------------------------------------------------------------------

  def store_model(self, model, file_name):
    '''
    This function stores the model pickle file to memory.

    Inputs:
      self
      model : model to store
      file_name : Name of the model pickly file

    Returns
      None
    '''
    pd.to_pickle(model, file_name)


In [ ]:
test = deployment_model_training(file_directory='../data/')
test.load_data(verbose = True)

Please wait: Loading the data into memory...
Data successfully loaded!
It took 10.49 seconds to load the data.


In [ ]:
train_df, test_df = test.preprocessing(verbose = True)

Handling Missing values and outliers......
Done with pre processing! Time Elapsed : 0.49 seconds
Creating new and better features......
Done with Feature Engineering! Time Elapsed : 3.62 seconds
(356251, 25)
(338857, 81)
The shape of Final training data : (307507, 105)
The shape of Final test data : (48744, 105)


In [ ]:
lgbm_model = test.lgbm_model(train_df, test_df, nfolds = 3)

Training until validation scores don't improve for 200 rounds.
[200]	training's binary_logloss: 0.252879	training's auc: 0.741828	valid_1's binary_logloss: 0.259711	valid_1's auc: 0.707623
[400]	training's binary_logloss: 0.24603	training's auc: 0.765186	valid_1's binary_logloss: 0.257598	valid_1's auc: 0.715205
[600]	training's binary_logloss: 0.241396	training's auc: 0.78141	valid_1's binary_logloss: 0.257099	valid_1's auc: 0.717085
[800]	training's binary_logloss: 0.237016	training's auc: 0.796292	valid_1's binary_logloss: 0.25679	valid_1's auc: 0.718073
[1000]	training's binary_logloss: 0.233167	training's auc: 0.808843	valid_1's binary_logloss: 0.256811	valid_1's auc: 0.718026
Early stopping, best iteration is:
[895]	training's binary_logloss: 0.235194	training's auc: 0.802311	valid_1's binary_logloss: 0.256763	valid_1's auc: 0.718137
Fold  1  TRAIN AUC :  0.8023108885635406
Fold  1  CV AUC :  0.7181371423199652
Training until validation scores don't improve for 200 rounds.
[200]	

In [ ]:
test.store_model(lgbm_model, 'lgbm_deployment_model.pkl')

In [ ]:
y = train_df.pop('TARGET')
X = train_df

fpr, tpr, threshold = roc_curve(y, lgbm_model.predict_proba(X)[:,1])
j_stat = tpr - fpr
best_threshold = threshold[np.argmax(j_stat)]
print(f"Best Threshold = {best_threshold}")

Best Threshold = 0.08347541315538703


In [ ]:
class final_submission:
  '''
  This class deals with the final submission for the case study.
  It contains 4 functions:

    1. init function for initialization
    2. Pre processing function for data pre processing
    3. final_function_1 takes a datapoint as input and gives a prediction as output. 
    4. final_function_2 takes datapoint(s) as input, target values and gives a metric as output (AUC score)
  '''
  
  # Function 1 : Initialization
  def __init__(self, model):
      '''
      Initialization Function
      '''
      self.model = model

  # ----------------------------------------------------------------------------------------------------------
  # Function 2 : Creating new features 
  def feature_engineering(self, app_data, prev_data):
      '''
      This function preprocesses the loaded data. The processing includes:

        4. Feature Engineering to create some better features

      Inputs:
        self
        app_data : application data
        prev_data : previous applications data
      
      Returns:
        final_datapoint : final_datapoint for testing
      '''
    
      # --------------------------------------------------------------------------------------------------------------

      # Task : Feature Engineering
      # --------------------------------------------------------------------------------------------------------------
      
      # i) Application_train/test Feature Engineering
      app_data['NEW_CREDIT_TO_ANNUITY_RATIO'] = app_data['AMT_CREDIT'] / app_data['AMT_ANNUITY']
      app_data['NEW_CREDIT_TO_GOODS_RATIO'] = app_data['AMT_CREDIT'] / app_data['AMT_GOODS_PRICE']
      app_data['INCOME_PER_PERSON'] = app_data['AMT_INCOME_TOTAL'] / app_data['CNT_FAM_MEMBERS']
      app_data['ANNUITY_INCOME_PERC'] = app_data['AMT_ANNUITY'] / app_data['AMT_INCOME_TOTAL']
      app_data['INCOME_CREDIT_PERC'] = app_data['AMT_INCOME_TOTAL'] / app_data['AMT_CREDIT']
      app_data['DAYS_EMPLOYED_PERC'] = app_data['YEARS_EMPLOYED'] / app_data['AGE']
      app_data['PAYMENT_RATE'] = app_data['AMT_ANNUITY'] / app_data['AMT_CREDIT']

      # Handling Categorical Data
      # 1. Gender
      if app_data['CODE_GENDER'] == 'F':
        app_data['CODE_GENDER_F'] = 1
        app_data['CODE_GENDER_M'] = 0
        app_data['CODE_GENDER_nan'] = 0
        app_data.drop(labels = ['CODE_GENDER'], inplace = True)

      elif app_data['CODE_GENDER'] == 'M':
        app_data['CODE_GENDER_F'] = 0
        app_data['CODE_GENDER_M'] = 1
        app_data['CODE_GENDER_nan'] = 0
        app_data.drop(labels = ['CODE_GENDER'], inplace = True)
      
      else:
        app_data['CODE_GENDER_F'] = 0
        app_data['CODE_GENDER_M'] = 0
        app_data['CODE_GENDER_nan'] = 1
        app_data.drop(labels = ['CODE_GENDER'], inplace = True)

      # 2. FLAG_OWN_CAR
      if app_data['FLAG_OWN_CAR'] == 'N':
        app_data['FLAG_OWN_CAR_N'] = 1
        app_data['FLAG_OWN_CAR_Y'] = 0
        app_data['FLAG_OWN_CAR_nan'] = 0
        app_data.drop(labels = ['FLAG_OWN_CAR'], inplace = True)
      
      elif app_data['FLAG_OWN_CAR'] == 'Y':
        app_data['FLAG_OWN_CAR_N'] = 0
        app_data['FLAG_OWN_CAR_Y'] = 1
        app_data['FLAG_OWN_CAR_nan'] = 0
        app_data.drop(labels = ['FLAG_OWN_CAR'], inplace = True)

      else:
        app_data['FLAG_OWN_CAR_N'] = 0
        app_data['FLAG_OWN_CAR_Y'] = 1
        app_data['FLAG_OWN_CAR_nan'] = 0
        app_data.drop(labels = ['FLAG_OWN_CAR'], inplace = True)


      # ii) Previous Application Data
      prev_data['APP_CREDIT_PERC'] = prev_data['AMT_APPLICATION'] / prev_data['AMT_CREDIT']

      # Handling Categorical Data : Only One Cat column : NAME_CONTRACT_STATUS
      prev_data['NAME_CONTRACT_STATUS_Approved'] = np.where(prev_data['NAME_CONTRACT_STATUS'] == 'Approved', 1, 0)
      prev_data['NAME_CONTRACT_STATUS_Refused'] = np.where(prev_data['NAME_CONTRACT_STATUS'] == 'Refused', 1, 0)
      prev_data['NAME_CONTRACT_STATUS_nan'] = np.where(prev_data['NAME_CONTRACT_STATUS'] == np.nan, 1, 0)
      prev_data.drop(columns = ['NAME_CONTRACT_STATUS'], inplace= True)

      # Aggregations we will perform 
      prev_app_num_agg = {
                      'AMT_ANNUITY': ['min', 'max', 'mean'],
                      'AMT_APPLICATION': ['min', 'max', 'mean'],
                      'AMT_CREDIT': ['min', 'max', 'mean'],
                      'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
                      'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
                      'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
                      'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
                      'DAYS_DECISION': ['min', 'max', 'mean'],
                      'CNT_PAYMENT': ['mean', 'sum']
                    }
      
      prev_data['ID'] = 1

      # Perform the aggregations based on the sk_id_curr
      prev_agg_features = prev_data.groupby('ID').agg(prev_app_num_agg)
      prev_agg_features.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg_features.columns.tolist()])

      # Previous Applications: Approved Applications - only numerical features
      approved_applications = prev_data[prev_data['NAME_CONTRACT_STATUS_Approved'] == 1]
      approved_agg_features = approved_applications.groupby('ID').agg(prev_app_num_agg)
      approved_agg_features.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg_features.columns.tolist()])
      prev_agg_features = prev_agg_features.join(approved_agg_features, how='left', on='ID')
      
      # Previous Applications: Refused Applications - only numerical features
      refused_applications = prev_data[prev_data['NAME_CONTRACT_STATUS_Refused'] == 1]
      refused_agg_features = refused_applications.groupby('ID').agg(prev_app_num_agg)
      refused_agg_features.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg_features.columns.tolist()])
      prev_agg_features = prev_agg_features.join(refused_agg_features, how='left', on='ID')

      return app_data, prev_agg_features.iloc[0]

  # -------------------------------------------------------------------------------------------------------------------------------------

  # Function 3 : Final Function 1

  def final_function_1(self, data):
      '''
      This function takes a datapoint as input and a prediction as output. 

      Inputs:
        self
        datapoint : Input datapoint
      
      Returns:
        pred : predicted value
      '''
      start_time = time()

      pred_values = np.zeros(len(data))
      
      pred_values = self.model.predict_proba(data)[:,1]
      predicted_label = np.where(pred_values > 0.0834754, 1, 0)
        
      #printing the results
      print("-" * 100)
      print(f"Predicted Probabilties for given Client(s) being Defaulter is/are: {np.round(pred_values, 4)}")
      print(f"The class label for given query point is: {predicted_label}")
      print(f"Total Time Taken for prediction = {time() - start_time}")
      print('-' * 100)

      return pred_values
  
  # ---------------------------------------------------------------------------------------------------------------------

  # Function 4 : Final Function 2
  def final_function_2(self, data, target):
        start_time = time()
        #making predictions
        pred_values = np.zeros(data.shape[0])
        pred_values = model.predict_proba(data)[:,1]

        print("-" * 100)
        print(f"Predicted Probabilties for given Client(s) being Defaulter is/are:\n{np.round(pred_values, 4)}")
        predicted_labels = np.where(pred_values > 0.083, 1, 0)
        print(f"\nThe predicted class labels are:\n{predicted_labels}")
        #if there are more than 1 input, printing the Metrics
        if len(data)>1:
            print('-' * 100)
            print("\nThe Performance Metrics are:\n")
            try:
                print(f"ROC-AUC Score = {roc_auc_score(target, pred_values)}")
            except:
                print("Cannot calculate ROC-AUC Score as the Test Datapoints have only 1 type of Class Labels present. Try with different Datapoints")
            print(f"Recall Score = {recall_score(target, predicted_labels)}")
        print(f"Total Time taken for prediction = {time() - start_time}")
        print("-" * 100)
